In [1]:
from datetime import datetime
print("This notebook is to test and demonstrate the spectral line fitting and file IO.\n"
	  "This version started 120324, last automatic datestamp:",datetime.now())

This notebook is to test and demonstrate the spectral line fitting and file IO.
This version started 120324, last automatic datestamp: 2024-12-20 14:38:14.428207


In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

import importlib, time, copy, os, sys, numpy as np, astropy.units as u
np.set_printoptions(linewidth=160)

from astropy.io import fits; import matplotlib.pyplot as plt; from matplotlib.gridspec import GridSpec
plt.rcParams.update({'font.size': 18,'figure.figsize':[15,12],'image.origin':'lower'}) # Make the fonts big enough for papers

base_path = '../'
sys.path.append(base_path+'linefit_modules/')
from util import get_mask_errs, get_spice_err
from skew_correction import skew_correct, deskew_linefit_window
from skew_parameter_search import search_shifts_mp, shift_holder, refine_points
from linefit_leastsquares import lsq_fitter
from linefit_storage import linefits

fitter = lsq_fitter

/tmp/ipykernel_2091013/1527855273.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [3]:
linelist = {'Ar VIII+S III 700':700.3, 'O III 703':702.9, 'O III 704':703.9, 'Mg IX 706':706.0,
			'O II 718':718.5, 'S IV 745':744.9, 'S IV 748':748.4, 'S IV 750':750.2,
			'O V 759':758.7, 'S IV+O V 759':759.4, 'O V 760':760.3, 'O V 762':762.0,
			'N IV 765':765.1, 'Ne VIII 770':770.4, 'Mg VIII 772':772.3, 'Ne VIII 780':780.3,
			'S V 786':786.5, 'O IV 787':787.7, 'O IV 790':790.1, 'Ly Gamma 972':972.5,
			'C III 977':977.0, 'O I +- Na VI 989':988.7, 'N III 990':989.8, 'N III 992':991.6,
			'H I (+ O I) 1025':1025.7, 'O I 1027':1027.4, 'O VI 1032':1031.9, 'C II 1036':1036.5,
			'O VI 1037':1037.6}

line_names = list(linelist.keys())
line_waves = [linelist[name] for name in line_names]

In [4]:
spice_data_dir = '/home/jplowman/research/solar-physics/data/spice/20220307/' # Will need to update this to location of data file
spice_L2_filename = 'solo_L2_spice-n-ras_20220307T030536_V22_100663723-000.fits'
spice_L2_fullname = os.path.join(spice_data_dir, spice_L2_filename)
win_name = 'O II 718 - Peak'

hdul = fits.open(spice_L2_fullname)
hdul.info()
spice_dat, spice_hdr = hdul[win_name].data[0], hdul[win_name].header
spice_dat = spice_dat.transpose([2,1,0]).astype(np.float32)
hdul.close()

# Manually estimated parameters of skew correction:
xlshift, ylshift = 1.6626831549412866, -1.7115244726176873

# Initial search pattern. Avoid checking 0 shift in either direction because that results in a
# sharper image and therefore more variance:
xs_quad1, ys_quad1 = np.array(np.meshgrid([-5.0,-3.0,-1.0],[-5.0,-3.0,-1.0])).transpose([0,2,1])
xs_quad2, ys_quad2 = np.array(np.meshgrid([ 1.0, 3.0, 5.0],[-5.0,-3.0,-1.0])).transpose([0,2,1])
xs_quad3, ys_quad3 = np.array(np.meshgrid([ 1.0, 3.0, 5.0],[ 1.0, 3.0, 5.0])).transpose([0,2,1])
xs_quad4, ys_quad4 = np.array(np.meshgrid([-5.0,-3.0,-1.0],[ 1.0, 3.0, 5.0])).transpose([0,2,1])
xl, xh, yl, yh = [-5,5,-5,5]

Filename: /home/jplowman/research/solar-physics/data/spice/20220307/solo_L2_spice-n-ras_20220307T030536_V22_100663723-000.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  Mg IX 706 - Peak    1 PrimaryHDU     418   (128, 834, 50, 1)   float32   
  1  O II 718 - Peak    1 ImageHDU       417   (128, 834, 50, 1)   float32   
  2  Ne VIII 770 / Mg VIII 772 (Merged)    1 ImageHDU       424   (128, 834, 66, 1)   float32   
  3  S V 786 / O IV 787 - Extend    1 ImageHDU       426   (128, 834, 58, 1)   float32   
  4  Ly Gamma 972 - Peak    1 ImageHDU       417   (128, 834, 36, 1)   float32   
  5  C III 977 (Merged)    1 ImageHDU       426   (128, 834, 42, 1)   float32   
  6  O VI 1032 - Peak    1 ImageHDU       417   (128, 834, 34, 1)   float32   
  7  VARIABLE_KEYWORDS    1 BinTableHDU    398   1R x 28C   [128D, 128I, 128I, 128I, 128I, 128I, 128E, 128E, 128E, 128E, 4I, 4I, 4I, 4I, 4J, 4J, 4J, 4J, 2944A, 128D, 128D, 50D, 50D, 66D, 58D, 36D, 42D, 34D]   
  8  WCSDVARR  

In [5]:
shift_vars = shift_holder(spice_dat, spice_hdr, line_waves, line_names, fitter.__name__)

In [6]:
shift_vars = search_shifts_mp(spice_dat, spice_hdr, xs_quad1, ys_quad1, line_waves, line_names, fitter, shift_vars=shift_vars)

-5.0 -3.0 0.8606972243584914
-1.0 -1.0 0.8357062491276005
-3.0 -3.0 0.8418580376401629
-5.0 -1.0 0.8586458512075458
-3.0 -1.0 0.8409201275539082
-3.0 -5.0 0.8482685195455772
-1.0 -3.0 0.8211218362366302
-1.0 -5.0 0.8250168741327454
-5.0 -5.0 0.8631941169894991


In [7]:
shift_vars = search_shifts_mp(spice_dat, spice_hdr, xs_quad2, ys_quad2, line_waves, line_names, fitter, shift_vars=shift_vars)

1.0 -3.0 0.8201737915892591
5.0 -5.0 0.8636212990700285
5.0 -1.0 0.8542334202344652
3.0 -3.0 0.8391850840318014
3.0 -5.0 0.8479118496780782
3.0 -1.0 0.8361074340601237
1.0 -5.0 0.8235405477948441
1.0 -1.0 0.8342263692793533
5.0 -3.0 0.8583798814407347


In [8]:
shift_vars = search_shifts_mp(spice_dat, spice_hdr, xs_quad3, ys_quad3, line_waves, line_names, fitter, shift_vars=shift_vars)

3.0 5.0 0.8538282327436579
5.0 1.0 0.8553043403733718
5.0 3.0 0.8604000088602474
1.0 3.0 0.8290219298693499
5.0 5.0 0.8688236428815441
3.0 3.0 0.8450698323757658
1.0 1.0 0.8383894081745283
1.0 5.0 0.8329229015367956
3.0 1.0 0.8407148771631086


In [9]:
shift_vars = search_shifts_mp(spice_dat, spice_hdr, xs_quad4, ys_quad4, line_waves, line_names, fitter, shift_vars=shift_vars)

-5.0 1.0 0.8593736674093987
-5.0 3.0 0.8623029457490041
-5.0 5.0 0.8675858033932056
-1.0 3.0 0.8280222689772524
-3.0 5.0 0.8530760680134222
-3.0 3.0 0.8469068355212058
-1.0 5.0 0.8325455473294622
-1.0 1.0 0.8400130528151253
-3.0 1.0 0.8425012512665426


In [ ]:
x_refine, y_refine = refine_points(shift_vars,[-5,5],[-5,5], 11, 11, 20)
shift_vars = search_shifts_mp(spice_dat, spice_hdr, x_refine, y_refine, line_waves, line_names, fitter, shift_vars=shift_vars)
shift_vars.save()

In [ ]:
x_refine, y_refine = refine_points(shift_vars,[-5,5],[-5,5], 81, 81, 20)
shift_vars = search_shifts_mp(spice_dat, spice_hdr, x_refine, y_refine, line_waves, line_names, fitter, shift_vars=shift_vars)
shift_vars.save()

In [ ]:
shift_vars2 = shift_holder(spice_dat, spice_hdr, line_waves, line_names, fitter.__name__)
shift_vars2.load() # Just to check the save/load functionality

In [ ]:
(np.array(list(shift_vars2.valdict.values()))-np.array(list(shift_vars.valdict.values()))).T

In [ ]:
# Reinterpolate the search results to a finer linear grid for ease of plotting:
from scipy.interpolate import RegularGridInterpolator, LinearNDInterpolator as lndi

xa = np.array(list(shift_vars.valdict.values()))[:,0]
ya = np.array(list(shift_vars.valdict.values()))[:,1]
dat = np.array(list(shift_vars.valdict.values()))[:,2]

include = (np.abs(xa) > 1.0e-5)*(np.abs(ya) > 1.0e-5)

nx_plot, ny_plot = 41, 41
xya = np.vstack([xa[include],ya[include]]).T
xa0,ya0 = np.array(np.meshgrid(np.linspace(xl,xh,nx_plot),np.linspace(yl,yh,ny_plot))).transpose([0,2,1])
dat_interp = lndi(xya, dat[include])(xa0,ya0)

dat_interp = lndi(xya, dat[include])(xa0,ya0)

In [ ]:
sort_interp = np.argsort(dat_interp.flatten())
xsort_interp = xa0.flatten()[sort_interp]
ysort_interp = ya0.flatten()[sort_interp]

xl2, xh2 = xl-0.5*(xh-xl)/(nx_plot-1), xh+0.5*(xh-xl)/(nx_plot-1)
yl2, yh2 = yl-0.5*(yh-yl)/(ny_plot-1), yh+0.5*(yh-yl)/(ny_plot-1)

labelstr = spice_hdr['DATE']+' '+spice_hdr['EXTNAME']
labelstr = labelstr.replace('-','').replace(':','').replace('  ','_').replace(' ','_')
print(labelstr)

plt.imshow(dat_interp.T, extent=[xl2, xh2, yl2, yh2],cmap=plt.get_cmap('gray'))
plt.plot(xa,ya,'P',markersize=10,linewidth=5)
plt.title(spice_hdr['DATE']+' '+spice_hdr['EXTNAME']+': xyshift='+str([xa[np.argmin(dat)], ya[np.argmin(dat)]]))
plt.xlabel('x shift (arcsecond/angstrom)')
plt.ylabel('y shift (arcsecond/angstrom)')
plt.legend(['Sampled points'])
plt.colorbar()
plt.savefig(os.path.join(shift_vars.save_dir,'varplot_'+labelstr+'.png'))